<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_rest_api_sierra_bulk_item_level_holds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="250"/>

# REST API: Bulk Item-level Holds

**This script here:** https://github.com/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_rest_api_sierra_bulk_item_level_holds.ipynb

**Additional instructions here:** https://cincinnatilibrary-my.sharepoint.com/:w:/g/personal/rebecca_quinones_chpl_org/EZ0sAeiIhqRGvrkHF-93jvABZpCouaEWzeH3OKWwqKZq4A

---

**Purpose:** for various collection-health purposes, this script can be used in conjunction with CollectionHQ to move groups of items from one location to another using the holds functionality.

**The script expects:**

* Excel workbook with 1 sheet having the following format:
  
  * In this example below, the script will place four item-level holds, and set the locations to the coorisponding pickup location--using their pickup location codes (Anderson, Blue Ash, Deer Park, Price Hill)

| barcode       | branch pickup location	|
|-------------- |------------------------	|
| A000055325526 | an |
| A000057896540 | ba |
| A000067386443 | dp |
| A000071676870 | pr |

* Patron record number with no leading text character, or trailing check-digit

  (e.g. ~p1234567a~ → 1234567)

  * Patron cards currently in use for this project are:
    
    * patron record number: `2512420` (patron barcode: `507026982`)

    * patron record number: `2512422` (patron barcode: `507036982`)

---

*Note*: The current 5-character pickup location codes can be found here:

https://github.com/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_rest_api_sierra_pickup_locations.ipynb

---

## 1. Enter REST-API Credentials

In order to be authorized to use the REST API, you must enter valid credientials.

 ↴ **Click on the play button below**, then when prompted, enter in the **`auth_string`** as your password:

In [ ]:
#@title
# <--- Click the play button to the left to get started
import time

# this is the helper script that contains some needed functions
!wget --quiet https://raw.githubusercontent.com/cincinnatilibrary/collection-analysis/8352bf7d9debf027e760e426c6d54a1df3aee77c/misc/chpl_helper.py --output-document=chpl_helper.py
!pip install -U rich > /dev/null
time.sleep(1)

# import the helper functions we just downloaded ...
import chpl_helper
from getpass import getpass, getuser
from google.colab import files
import pandas as pd
import requests
import json
from io import StringIO
from rich.progress import Progress, SpinnerColumn, TextColumn, track
from rich import print
import logging

# create log file
ts = pd.Timestamp('now').isoformat()
logging.basicConfig(
    filename="bulk_holds_log.txt", 
    format="%(asctime)s\t%(levelname)s\t%(message)s", 
    level=logging.DEBUG
)
logging.info('script started: {}'.format(ts))

# point this to the base URL for the Sierra REST API
base_url = 'https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/'

# client_key = getpass('Enter the client KEY value: ')
# client_secret = getpass('Enter the client SECRET value: ')
auth_string = getpass('Enter the auth_string value: ')

# test our credentials

try:
  headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)
  r = requests.get(base_url + 'info/token', headers=headers, verify=True)
  print('access token expires in: {} seconds'.format(r.json()['expiresIn']))
  logging.info('keyId: {}'.format(r.json()['keyId']))
  logging.info('expiresIn: {}'.format(r.json()['expiresIn']))
except:
  logging.error('requst info/token status:{} \tcontent:'.format(r.status_code, r.content))


## 2. Upload Input File

This file will be processed to placed the holds.

 ↴ **Click on the play button below**, then when prompted, upload the input file:

In [ ]:
#@title
input_file = files.upload()

# convert the input file to a Dataframe ... 
df = pd.read_excel( 
    [key for key in input_file.keys()][0],
    usecols=[0,1],
    converters={
        0: str,
        1: str
    }
)
# print(df.head(5))
# print('...')
print('total rows in input: ', df.shape[0])

# construct the item query we'll use to convert barcodes to item record numbers 
# (API endpoint URLs)
operands = [row['barcode'] for i, row in df.iterrows()]
item_query = {
  "queries": [
    {
      "target": {
        "record": {
          "type": "item"
        },
        "field": {
          "tag": "b"
        }
      },
      "expr": [
        {
          "op": "in",
          "operands": operands
        }
      ]
    }
  ]
}

# print(json.dumps(item_query))

headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)

try:
  text_column = TextColumn("Searching")
  spinner_column = SpinnerColumn(finished_text='Complete!')
  with Progress(text_column, spinner_column, expand=False) as progress:
  #Progress(transient=True) as progress:
    task = progress.add_task("Searching ...")

    r = requests.post(
      base_url + 'items/query?offset=0&limit={}'.format(len(operands)), 
      headers=headers,
      data=json.dumps(item_query),
      verify=True
    )
    progress.update(task, completed=1)

  # print("response json: ", r.json())
  print('...')
  msg = 'total rows of items returned from search: {}'.format(r.json()['total'])
  logging.info(msg)
  print(msg)

except:
  logging.error('items/query failed')
  print('error?')

## 3. Enter the patron record number being use to place the holds:

When prompted, enter the patron record number without leading character, or check-digits

(e.g. ~p2509457a~ → 2509457):

↴ **Click on the play button below**

In [ ]:
#@title
patron_record_num = input('patron record number: ')

logging.info('using patron number {}'.format(patron_record_num))

try:
  r_patron = requests.get(
        base_url + 'patrons/{}/holds'.format(patron_record_num), 
        headers=headers,
        verify=True
      )
  logging.info(r_patron.json())
  print('patron {} has {} total holds'.format(
      patron_record_num, 
      r_patron.json()['total'])
  )
except:
  logging.error('patrons/{}/holds failed'.format(patron_record_num))
  logging.error(r_patron.content)

# testing if a item has volume records associated with it
#
# r_entry = requests.get(
#         url=r.json()['entries'][0]['link'] + '?fields=id,location,status,barcode,bibIds,volumes',
#         headers=headers,
#         verify=True
# )

# print(r_entry.json())

# if len(r_entry.json()['volumes']) > 0:
#   print('has vols')

## 4. Place the Holds:

 ↴ **Click on the play button below**, for the holds to be placed
 :

In [ ]:
#@title
len_entries = len(r.json()['entries'])
text_column = TextColumn("Placing Holds")
# spinner_column = SpinnerColumn(finished_text='Complete!')
with Progress(text_column, *Progress.get_default_columns(), expand=False) as progress:
  for i in progress.track(range(len_entries)):
    entry = r.json()['entries'][i]
    # print(i, entry['link'])
    r_entry = requests.get(
        url=entry['link'] + '?fields=id,location,status,barcode,bibIds,volumes',
        headers=headers,
        verify=True
    )
    
    # extract the item record number
    item_record_num = r_entry.json()['id']
        
    #extract the barcode
    search_barcode = str(r_entry.json()['barcode'])
    
    # NOTE: this patron record was previously used for testing
    # https://classic.cincinnatilibrary.org/iii/sierra-api/v6/patrons/2509457
    # patron_record_num = '2509457'

    # match the barcode to get the pickup location code in the input file / Dataframe 
    pickupLocation = df[df.barcode == search_barcode]['branch pickup location'].values[0]
    # patron_record_num = df[df.barcode == search_barcode].branch_account.values[0].strip()[1:-1]
    
    # print('placing hold...')

    logging.info(f"item_record_num\t{item_record_num}\tsearch_barcode\t{search_barcode}\tpatron_record_num\t{patron_record_num}")
    
    # place the hold ...    
    body_data = {
        "recordType": "i",
        "recordNumber": int(item_record_num),
        "pickupLocation": pickupLocation,
        # "neededBy": "",
        # "numberOfCopies": 1,
        "note": "chpl-bulk-hold-script"
    }

    logging.info(f"patrons/{patron_record_num}/holds/requests {json.dumps(body_data)}")
    
    r_hold = requests.post(
        url=base_url + f'patrons/{patron_record_num}/holds/requests',
        headers=headers,
        data=json.dumps(body_data),
        verify=True
    )

    # print("response json: ", r_hold.json())
    logging.info(f"response\t{r_hold.status_code}\t{r_hold.content}")



## 4a. Download log file:

 ↩ **Click on the "Files" button (image.png) to the left, right the file `bulk_holds_log.txt` and then click `Download`**

## 5. Remove the Holds:

If you wish to remove all holds for a patron record num, you can do so here.

 ↴ **Click on the play button below**, input the patron record num, and ALL the holds will be removed for that patron

In [ ]:
#@title
#  delete /v6/patrons/{id}/holds 

patron_record_num = 0
patron_record_num = input('patron record number: ')

# cancel all the holds for this patron account
r_cancel = requests.delete(
        url=base_url + f'patrons/{patron_record_num}/holds',
        headers=headers,
        verify=True
)

print('response:\n', r_cancel.status_code, '\n', r_cancel.content, sep='')